# Hello

Welcome to the SME Pavement Crack Detection Interactive Notebook.

# Install Dependencies

Uncomment and run the first time to make sure you have all the dependencies.

In [ ]:
#!pip install -r dependencies.txt

# Imports

 Please run the following code to import all of the functions.

In [ ]:
import numpy as np
import os
from skimage import io
from skimage.filters import threshold_multiotsu
from src.CreatePaths import createPaths
from src.LoadImage import loadImage
from src.ThresholdCutImages import thresholdCutImages
from src.HeatImage import heatImage
from src.TestImages import loadLabeledImages, testImages
from src.SaveDictionary import saveDictionary
from src.StitchImageDict import stitchImageDict, stitchRows

# Globals

The following are definitions of global paramters. Change them as need be, but best to leave them alone.

In [ ]:
# Feel free to change
DIVIDE_X = 25  # Number of divisions on one of axies
DIVIDE_Y = 25  # Number of diisions on other axis

# Try not to change
dir_path = os.path.realpath('.')  # The directory of the project.

# Do not change
thresholding_function = threshold_multiotsu  # Function for thresholding.


# Creating Paths

The following code is to create the different sub folders that will be required to run the rest of the code.

In [ ]:

required_paths = [  # The folders that must exist.
    os.path.realpath("./data"),
    os.path.realpath("./data/raw_images"),  # Where the .tif files will be stored.
    os.path.realpath("./results"),
]
createPaths(required_paths, dir_path)

# STOP

Before proceding, please place your image in the `data/raw_images` folder.

In [ ]:
IMAGES = os.listdir(os.path.join(dir_path , "data" , "raw_images"))

In [ ]:
image_dict = {}
for image in IMAGES:
    # Name the image
    image_dict[image] = {}
    image_dict[image]["file_name"] = image  # Redundent for the sake of clarity
    image_dict[image]["name"] = image.split('.')[0]  # Without the .tif
    
    # Name the paths
    image_dict[image]["raw_path"] = os.path.join("data" , "raw_images", image_dict[image]["file_name"])
    image_dict[image]["cut_path"] = os.path.join("results" , image_dict[image]["name"], "raw_cut_images")
    image_dict[image]["thresholded_cut_path"] = os.path.join("results" ,  image_dict[image]["name"] , "thresholded_cut_images")
    image_dict[image]["labeled_cut_path"] = os.path.join("results" ,  image_dict[image]["name"] , "labeled_cut_images")
    image_dict[image]["stitched_path"] = os.path.join("results" ,  image_dict[image]["name"] , "stitched_image")

In [ ]:
for key in image_dict:
    required_paths.append(image_dict[image]["cut_path"])
    required_paths.append(image_dict[image]["thresholded_cut_path"])
    required_paths.append(image_dict[image]["labeled_cut_path"])
    required_paths.append(image_dict[key]["stitched_path"])
createPaths(required_paths, dir_path)

# Cut and Threshold Images

Running the cell below will cut and threshold the images insdie of the `data/raw_images` folder.

In [ ]:
image_dict

In [ ]:
for key in image_dict:
    image_dict[key]["cut_image_dict"] = loadImage(image_dict[key]["raw_path"], DIVIDE_X, DIVIDE_Y)
    image_dict[key]["thresholded_dict"] = thresholdCutImages(image_dict[key]["cut_image_dict"], thresholding_function)

In [ ]:
image_dict[IMAGES[0]].keys()

After running the cell above, the image has now been cut and thresholded. You may want to run the cell below in order to save the cut images and the thresholded images in their respective paths.

In [ ]:
# Saves the cut images
saveDictionary(image_dict[key]["cut_image_dict"], dir_path, image_dict[key]["cut_path"], "cut_image")

In [ ]:
saveDictionary(image_dict[key]["thresholded_dict"], dir_path, image_dict[key]["thresholded_cut_path"], "cut_image")

# Stitch Image

In [ ]:
for key in image_dict:
    image_dict[key]["stitched_image"] = stitchImageDict(image_dict[key]["thresholded_dict"])


In [ ]:
for key in image_dict:
    io.imsave(os.path.join(dir_path, image_dict[key]['stitched_path'], "stitched_image.tif" ), image_dict[key]["stitched_image"])
    

# Create heatmap

In [ ]:
for key in image_dict:
    image_dict[key]["heat"] = heatImage(image_dict[key]["thresholded_dict"])

In [ ]:
heatmap = np.zeros((25,25))
for another_key in image_dict[key]["heat"]:
    heatmap[another_key[0],another_key[1]] = image_dict[key]["heat"][another_key]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(heatmap)